In [1]:
import os
import requests
import unicodedata
import re
import time

In [2]:
cwd = os.getcwd()
path = os.path.dirname(os.path.dirname(cwd))
os.chdir(path)

In [3]:
os.getcwd()

'/home/nacho/Documents/CESP/SPADE_mutiagents'

In [4]:
#If there is no such folder, the script will create one automatically
folder_location = 'dataset_stjjalisco_sentencias'
if not os.path.exists(folder_location):os.mkdir(folder_location)
os.chdir(folder_location)
os.getcwd()

'/home/nacho/Documents/CESP/SPADE_mutiagents/dataset_stjjalisco_sentencias'

In [5]:
def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

In [6]:
page_number = 1
url = "https://publica-sentencias-backend.stjjalisco.gob.mx/tocas?page=" + str(page_number) + "&sala_id=&numero=&periodo=&derivado=&delito_id=&materia_id=&nombre_delito=&accion_especifica_id=&accion_simultanea_id="
page = requests.get(url)

In [7]:
page.json()['data']['tocas'].keys()

dict_keys(['current_page', 'data', 'first_page_url', 'from', 'last_page', 'last_page_url', 'next_page_url', 'path', 'per_page', 'prev_page_url', 'to', 'total'])

In [8]:
for toca in page.json()['data']['tocas']['data']:
    print(toca["delito_data"]["nombre"])

LESIONES A TITULO DE CULPA
ACCIÓN DE PAGO
LESIONES A TITULO DE CULPA Y DAÑO EN LAS COSAS A TITULO DE CULPA
EJECUCIÓN DE FIDEICOMISO
ACCIÓN DE PAGO
ACCIÓN DE FIJACIÓN DE VISITAS Y CONVIVENCIA
NULIDAD TESTAMENTO
REVOCACIÓN DE DONACIÓN
ACCIÓN DE CUMPLIMIENTO DE CONTRATO
REAL HIPOTECARIA 
ACCIÓN DE DAÑO MORAL
VENCIMIENTO ANTICIPADO DEL CONTRATO
ACCIÓN DE CUMPLIMIENTO DE CONTRATO
ACCIÓN DE NULIDAD
ACCIÓN REIVINDICATORIA


In [ ]:
last_page = page.json()['data']['tocas']['last_page']
for page_number in range(1, last_page + 1):
    page = requests.get("https://publica-sentencias-backend.stjjalisco.gob.mx/tocas?page=" + str(page_number))
    if page.status_code == 429:
        time.sleep(int(page.headers['retry-after']))
    else:
        for toca in page.json()['data']['tocas']['data']:
            try:
                toca_file_response = requests.get("https://publica-sentencias-backend.stjjalisco.gob.mx/toca/" + str(toca["id"]) + "/file")
                if toca["accion_especifica_data"] is None and toca["accion_simultanea_data"] is None:
                    toca_file_filename = toca["fecha_publicacion"] + str("_") + toca["materia_data"]["nombre"] + str("_") + toca["delito_data"]["nombre"]
                elif toca["accion_especifica_data"] is not None and toca["accion_simultanea_data"] is not None:
                    toca_file_filename = toca["fecha_publicacion"] + str("_") + toca["materia_data"]["nombre"] + str("_") + toca["delito_data"]["nombre"] + str("_") + toca["accion_especifica_data"]['nombre'] + str("_") + toca["accion_simultanea_data"]['nombre']
                elif toca["accion_especifica_data"] is not None and toca["accion_simultanea_data"] is None:
                    toca_file_filename = toca["fecha_publicacion"] + str("_") + toca["materia_data"]["nombre"] + str("_") + toca["delito_data"]["nombre"] + str("_") + toca["accion_especifica_data"]['nombre']
                elif toca["accion_especifica_data"] is None and toca["accion_simultanea_data"] is not None:
                    toca_file_filename = toca["fecha_publicacion"] + str("_") + toca["materia_data"]["nombre"] + str("_") + toca["delito_data"]["nombre"] + str("_") + toca["accion_simultanea_data"]['nombre']
            except TypeError:
                pass
            finally:
                with open(slugify(toca_file_filename) + '.pdf', 'wb') as f:
                        f.write(toca_file_response.content)

In [ ]:
last_page = page.json()['data']['tocas']['last_page']
for page_number in range(1, last_page + 1):
    page = requests.get("https://publica-sentencias-backend.stjjalisco.gob.mx/tocas?page=" + str(page_number))
    if page.status_code == 429:
        time.sleep(int(page.headers['retry-after']))
    else:
        for toca in page.json()['data']['tocas']['data']:
            try:
                if toca["delito_data"]["nombre"] == 'FEMINICIDIO':
                # if toca["materia_data"]["nombre"] == 'Penal':
                    toca_file_response = requests.get("https://publica-sentencias-backend.stjjalisco.gob.mx/toca/" + str(toca["id"]) + "/file")
                    if toca["accion_especifica_data_id"] is None and toca["accion_simultanea_data_id"] is None:
                        toca_file_filename = toca["fecha_publicacion"] + str("_") + toca["materia_data"]["nombre"] + str("_") + toca["delito_data"]["nombre"]
                    elif toca["accion_especifica_data_id"] is not None and toca["accion_simultanea_data_id"] is not None:
                        toca_file_filename = toca["fecha_publicacion"] + str("_") + toca["materia_data"]["nombre"] + str("_") + toca["delito_data"]["nombre"] + str("_") + toca["accion_especifica_data_id"]['nombre'] + str("_") + toca["accion_simultanea_data_id"]['nombre']
                    elif toca["accion_especifica_data_id"] is not None and toca["accion_simultanea_data_id"] is None:
                        toca_file_filename = toca["fecha_publicacion"] + str("_") + toca["materia_data"]["nombre"] + str("_") + toca["delito_data"]["nombre"] + str("_") + toca["accion_especifica_data_id"]['nombre']
                    elif toca["accion_especifica_data_id"] is None and toca["accion_simultanea_data_id"] is not None:
                        toca_file_filename = toca["fecha_publicacion"] + str("_") + toca["materia_data"]["nombre"] + str("_") + toca["delito_data"]["nombre"] + str("_") + toca["accion_simultanea_data_id"]['nombre']
            except TypeError:
                pass
            finally:
                with open(slugify(toca_file_filename) + '.pdf', 'wb') as f:
                    f.write(toca_file_response.content)